### Zonal statistics on ERA5 weather data
- Objectives: 
    - This notebook conducts zonal statistics on Indian weather data to aggregate the data at district level.
    - This is done in order to aggregate weather data at the same level as district-level rice yields for ML.

Import modules

In [1]:
# Import necessary libraries
import xarray as xr
import rioxarray
import geopandas as gpd
import rasterio as rio
import matplotlib.pyplot as plt
import regionmask

Load data

In [2]:
# Load India shapefile
india_shapefile = gpd.read_file(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\SHAPEFILE\gadm36_IND_0.shp")

# Load India districts shapefile
india_districts_shapefile = gpd.read_file(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\SHAPEFILE\gadm36_IND_2.shp")

In [52]:
# Load weather data (.nc format)
climate_data = xr.open_dataset(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\CDS\weather_data.nc")

# Convert temperature to celsius  
climate_data['t2m'] = climate_data['t2m'] - 273.15

Apply zonal statistics

In [64]:
# 1. Create mask from shapefile and xarray climate_data
mask = regionmask.from_geopandas(india_districts_shapefile, 
                                 names="GID_2", 
                                 abbrevs="_from_name").mask(climate_data['longitude'],
                                                            climate_data['latitude'])

In [74]:
# 2. Aggregate variables in the xarray using the mask
regional_agg_dr = climate_data.groupby(mask).mean() # max, min, mapping custom functions also work

In [75]:
# 3. Convert the aggregated xarray to a dataframe
regional_agg = regional_agg_dr.to_dataframe()
regional_agg.reset_index(inplace=True)

In [76]:
# add region information and drop mask column
regional_agg['GID_2'] = regional_agg.apply(lambda x: india_districts_shapefile['GID_2'][int(x['mask'])], axis=1)
regional_agg = regional_agg.drop(columns=['mask'])
# regional_agg = regional_agg.drop(columns=['crs'])

In [77]:
# Query to keep data up to end of 2022 and keep only expver 1
regional_agg = regional_agg.query('expver == 1')
regional_agg = regional_agg.query('time < "2023-01-01"')

In [78]:
# order dataset by 'tp' variable from biggest to smallest
regional_agg

,time,expver,t2m,tp,GID_2
0,2000-01-01,1,25.948486,0.000254,IND.1.2_1
2,2000-02-01,1,25.903374,0.001822,IND.1.2_1
4,2000-03-01,1,26.487930,0.005368,IND.1.2_1
6,2000-04-01,1,27.429024,0.005059,IND.1.2_1
8,2000-05-01,1,27.479309,0.009505,IND.1.2_1
...,...,...,...,...,...
371198,2022-08-01,1,30.092602,0.007680,IND.36.20_1
371200,2022-09-01,1,28.889685,0.011895,IND.36.20_1
371202,2022-10-01,1,26.727991,0.005235,IND.36.20_1
371204,2022-11-01,1,22.756390,0.000031,IND.36.20_1


Save data

In [80]:
# merge to india districts shapefile
india_districts_weather = india_districts_shapefile.merge(regional_agg, on='GID_2')

In [82]:
# Keep only the columns GID_0, NAME_0, GID_1, NAME_1, GID_2, NAME_2, time, t2m, tp, region
india_districts_weather = india_districts_weather[['GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'GID_2', 'NAME_2', 'time', 't2m', 'tp']]

In [84]:
india_districts_weather.head()

,GID_0,NAME_0,GID_1,NAME_1,GID_2,NAME_2,time,t2m,tp
0,IND,India,IND.1_1,Andaman and Nicobar,IND.1.2_1,North and Middle Andaman,2000-01-01,25.948486,0.000254
1,IND,India,IND.1_1,Andaman and Nicobar,IND.1.2_1,North and Middle Andaman,2000-02-01,25.903374,0.001822
2,IND,India,IND.1_1,Andaman and Nicobar,IND.1.2_1,North and Middle Andaman,2000-03-01,26.487930,0.005368
3,IND,India,IND.1_1,Andaman and Nicobar,IND.1.2_1,North and Middle Andaman,2000-04-01,27.429024,0.005059
4,IND,India,IND.1_1,Andaman and Nicobar,IND.1.2_1,North and Middle Andaman,2000-05-01,27.479309,0.009505


In [85]:
# save as excel
india_districts_weather.to_excel(r"..\4_data\PROCESSED_DATA\WEATHER\india_districts_weather_data.xlsx", index=False)

Alternative method

In [ ]:
# # Create mask of multiple regions from shapefile
# india_mask = regionmask.mask_3D_geopandas(
#         india_districts_shapefile,
#         climate_data.longitude,
#         climate_data.latitude,
#         drop=True,
#         numbers="index"
#     )
# # Apply the mask to the xarray data
# climate_data_masked = climate_data.where(india_mask)
# # Calculate mean values per district, per time step
# mean_values = climate_data_masked.groupby('time').mean(dim=['latitude', 'longitude'])
# # Convert values to dataframe
# df_mean_values = mean_values.to_dataframe().reset_index()
# # Query to keep data up to end of 2022 and keep only expver 1
# df_mean_values = df_mean_values.query('expver == 1')
# df_mean_values = df_mean_values.query('time < "2023-01-01"')
# # join to shapefile, df_mean_values region matches to shapefile index column
# india_districts_shapefile = india_districts_shapefile.merge(df_mean_values, left_on='index', right_on='region')
# india_districts_shapefile